<a href="https://colab.research.google.com/github/carlibeisel/mason_water_budget/blob/main/reach_precip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

By Carli Beisel

Created August, 2024

Purpose: Calculates precipitation for each Reach within Mason Drainage from Daymet.

## 1. Install packages and connect to Google Drive

In [ ]:
# Installs geemap package
import subprocess
!pip install geemap

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project = 'extract-gridmet')

In [ ]:
!pip install geopandas
import geopandas as gpd #import independent shapefile
import json #for metadata of shapefile
import os #for file paths
import numpy as np #for stats and arrays
import pandas as pd #for dataframes
!pip install pycrs
import pycrs

#Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

2. Import reach shapefiles and start/end dates to clip to dataset

In [ ]:
## ------------------------------------------------------- ##
##   Import shapefile and start/end dates to clip dataset  ##
## ------------------------------------------------------- ##
start_end = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_input/start_end_dates.csv')
shp_file = '/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_input/drain_delineation/Drains_Merge_07072022.shp'
subset = emap.shp_to_ee(shp_file) # converts shapefile to feature in GEE

map = emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(ee.Image().paint(subset, 0, 2), {}, 'POU')
map.addLayerControl()
map

In [ ]:
## ------------------------------------ ##
##        REACH PRECIPITATION           ##
## ------------------------------------ ##

# NOTE: wy = Water Year: October 1 - September 30

# CHANGE TO DAILY SUM!
years = np.arange(1987,2021)
wy_pr = []

for i in range(len(start_end)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i]-1)+'-10-01'), (str(years[i])+'-9-30')) #get image collection for water year
  pr = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index': str(start_end['Year'][i])}) #select precip to analyze and sum
  wy_pr.append(pr)

wy_pr = ee.ImageCollection(wy_pr)

In [ ]:
## ------------------------ ##
##   CALCULATE ZONAL STATS  ##
## ------------------------ ##

# Change to daily sum!

out_stats = os.path.join('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/wy_precip_stats.csv')
emap.zonal_statistics(wy_pr, subset, out_stats, statistics_type='MEAN', scale=1000)

In [ ]:
## ---------------------------------------------- ##
##   CREATE CLIMATE STAT FOR EACH POU AND EXPORT  ##
## ---------------------------------------------- ##

# delete excess and make for each reach

years = np.arange(1987,2021)
wy_precip = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/wy_precip_stats.csv')
ant_precip = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/ant_precip_stats.csv')
JA_temp = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/JAtemp_stats.csv')
irrig_temp = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/ir_tmp_stats.csv')
et_irrig = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/et.csv')


names = et_irrig['Name']

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['NAME'] = names[i]
  df['ant_prcp'] = ant_precip.iloc[i,0:34].values
  df['wy_prcp'] = wy_precip.iloc[i,0:34].values
  df['irrig_temp'] = irrig_temp.iloc[i,0:34].values
  df['JuneAug_temp'] = JA_temp.iloc[i,0:34].values
  df['et'] = et_irrig.iloc[i,0:34].values
  out_path = os.path.join('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/final/'+names[i]+'_climate.csv')
  df.to_csv(out_path)